In [1]:
from copy import deepcopy
from pathlib import Path
import os
import re
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_parse import LlamaParse
from llama_index.core.schema import TextNode
from typing import Optional


In [2]:
import nest_asyncio

nest_asyncio.apply()

### Defining Pydantic Models for Structured Output

In this section, we define several Pydantic models to structure the output data. These models ensure that the extracted and generated data adheres to a predefined schema, facilitating validation and consistency.

- `KnowledgeStatement`: Represents a knowledge statement with an ID and text.
- `AbilityStatement`: Represents an ability statement with an ID and text.
- `Topic`: Represents a topic with a name, subtopics, knowledge statements, and ability statements.
- `LearningUnit`: Represents a learning unit with a name, topics, and a learning outcome.
- `AssessmentMethod`: Represents an assessment method with a code and duration.
- `FacilitatorGuideExtraction`: Represents the structured data extracted from the facilitator guide, including course title, proficiency level, learning units, and assessments.
- `KnowledgeStatementContent`: Represents the content retrieved for a knowledge statement, including the knowledge ID, statement, topic name, and retrieved content.
- `WSQ`: Represents a workplace scenario question, including the knowledge ID, statement, scenario, question, and answer.
- `CaseStudyQuestion`: Represents a case study question with a question, answer, and associated abilities.
- `CaseStudy`: Represents a case study with a scenario and a list of case study questions.

These models are used to structure and validate the data throughout the extraction and generation processes, ensuring that the output is consistent and reliable.

In [3]:
from llama_index.llms.openai import OpenAI
from pydantic.v1 import BaseModel, Field
from typing import List

# Define Pydantic models for structured output
class KnowledgeStatement(BaseModel):
    id: str
    text: str


class AbilityStatement(BaseModel):
    id: str
    text: str


class Topic(BaseModel):
    name: str
    subtopics: List[str]
    tsc_knowledges: List[KnowledgeStatement]
    tsc_abilities: List[AbilityStatement]


class LearningUnit(BaseModel):
    name: str
    topics: List[Topic]
    learning_outcome: str


class AssessmentMethod(BaseModel):
    code: str
    duration: str

class FacilitatorGuideExtraction(BaseModel):
    course_title: str
    tsc_proficiency_level: str
    learning_units: List[LearningUnit]
    assessments: List[AssessmentMethod]  # New field for assessments

# Define a Pydantic model for the Knowledge Statement and its Retrieved Content
class KnowledgeStatementContent(BaseModel):
    knowledge_id: str = Field(..., description="The ID of the Knowledge Statement, e.g., K1, K2.")
    knowledge_statement: str = Field(..., description="The text of the Knowledge Statement.")
    topic_name: str = Field(..., description="The name of the topic associated with this Knowledge Statement.")
    retrieved_content: str = Field(..., description="The content retrieved for this Knowledge Statement.")

# Define the WSQ model for structured output
class WSQ(BaseModel):
    knowledge_id: str = Field(..., description="The ID of the Knowledge Statement, e.g., K1, K2.")
    knowledge_statement: str = Field(..., description="The text of the Knowledge Statement.")
    scenario: str = Field(..., description="The realistic workplace scenario.")
    question: str = Field(..., description="The question based on the scenario.")
    answer: str = Field(..., description="The concise answer to the question.")

class LearningOutcomeContent(BaseModel):
    ability_id: List[str]
    retrieved_content: str = Field(..., description="The content retrieved for this Knowledge Statement.")

## Setting Up LlamaParse, LlamaIndex and OpenAI Models

In this section, we initialize the `Settings` for LlamaIndex with the OpenAI embedding and language models. We use the `OpenAIEmbedding` model for embeddings and the `OpenAI` model for language processing.

In [4]:
from llama_index.llms.openai import OpenAI 
from llama_index.core.schema import Document


In [5]:
OPENAI_API_KEY = os.getenv('TERTIARY_INFOTECH_API_KEY') 
LLAMA_API_KEY = os.getenv('LLAMA_CLOUD_API_KEY')

In [6]:
embed_model = OpenAIEmbedding(model="text-embedding-3-large", api_key=OPENAI_API_KEY)
llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

Settings.embed_model = embed_model
Settings.llm = llm

In [35]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model="text-embedding-3-large", api_key=OPENAI_API_KEY)
llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

Settings.llm = llm
Settings.embed_model = embed_model

In [36]:
from llama_index.core.node_parser import LlamaParseJsonNodeParser
from llama_index.core import VectorStoreIndex
slides_parser = LlamaParse(
    api_key=LLAMA_API_KEY,
    result_type="text",
    fast_mode=True,
    num_workers=8,
    verbose=True,
)
file_path = "../input/WSQ- Learner Guide Slides - Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini - v5.pdf"
md_json_objs = slides_parser.get_json_result(file_path)

Started parsing the file under job_id 90655fac-68db-4358-a78d-d3240b3713ea


In [37]:
md_json_list = md_json_objs[0]["pages"] 

In [49]:
slides_parser = LlamaParse(
    api_key=LLAMA_API_KEY,
    # Use "markdown" or "json" because we're calling get_json_result()
    result_type="markdown",
    num_workers=8,
    verbose=True,
)

json_objs = slides_parser.get_json_result(file_path)
json_list = json_objs[0]["pages"]

documents = []
for page in json_list:
    documents.append(
        Document(
            text=page.get("text", ""),
            metadata=page,
        )
    )

node_parser = LlamaParseJsonNodeParser(
    llm=OpenAI(model="gpt-4o-mini"),
    num_workers=8,
    include_metadata=True
)

nodes = node_parser.get_nodes_from_documents(documents)
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

index = VectorStoreIndex(nodes=base_nodes + objects)

Started parsing the file under job_id 6c7a9703-a68c-4060-af5b-b6b94f943507


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, ?it/s]
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.1490642624376317 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.44660132977111155 seconds 

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [48]:
print(nodes)

[]


In [38]:
from llama_index.core.schema import BaseNode, TextNode, Document

documents = []
for _, page in enumerate(md_json_list):
    documents.append(
        Document(
            text=page.get("text"),
            metadata=page,
        )
    )

for doc in documents:
    print(doc.id_, doc.text.strip()[:10], doc.metadata)
    break

099d364a-d12e-4ee8-8d7a-b3758d70aa88 Develop Ar {'page': 1, 'text': ' Develop Artificial Intelligence and Large\nLanguage Model (LLM) Applications withGoogle Gemini\n                         Semini\n  WS     SINGAPORE\n         WorkforCE SKILL8\n         QUALIFICATIONS                                                      Version: 5.0\n                                                                     Learner Guide Slides\n  T      Umfmtech                                      Website:www.tertiarycourses.com.sg\n                                                       Email: enquiry@tertiaryinfotech.comCourse Code: TGS-2024042961\n                                                                      Tel: +65 6100 0613\n                     This material belongs to Tertiary Infotech Pte Ltd (UEN: 20120096W). All Rights Reserved', 'images': [{'name': 'img_p0_3.png', 'height': 137, 'width': 300, 'x': 15.348425687999999, 'y': 219.125947956912, 'original_width': 300, 'original_height': 137, 

In [39]:
print(len(documents))


216


In [46]:
print(documents[0].text)

 Develop Artificial Intelligence and Large
Language Model (LLM) Applications withGoogle Gemini
                         Semini
  WS     SINGAPORE
         WorkforCE SKILL8
         QUALIFICATIONS                                                      Version: 5.0
                                                                     Learner Guide Slides
  T      Umfmtech                                      Website:www.tertiarycourses.com.sg
                                                       Email: enquiry@tertiaryinfotech.comCourse Code: TGS-2024042961
                                                                      Tel: +65 6100 0613
                     This material belongs to Tertiary Infotech Pte Ltd (UEN: 20120096W). All Rights Reserved


In [40]:
from llama_index.core.node_parser import LlamaParseJsonNodeParser
from llama_index.llms.openai import OpenAI

node_parser = LlamaParseJsonNodeParser(
    llm=OpenAI(model="gpt-4o-mini"), num_workers=16, include_metadata=True
)
nodes = node_parser.get_nodes_from_documents(documents)
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, 

In [45]:
print(nodes)

[]


In [43]:
from llama_index.core import VectorStoreIndex


recursive_index = VectorStoreIndex(nodes=base_nodes + objects)
recursive_query_engine = recursive_index.as_query_engine(
    similarity_top_k=5, verbose=True
)
res = recursive_query_engine.query(
    "what is MultiModal"
)

In [44]:
print(res)

Empty Response


In [17]:
print(md_json_list[1].keys())

dict_keys(['page', 'text', 'images', 'charts', 'items', 'status', 'links', 'width', 'height', 'triggeredAutoMode', 'structuredData', 'noStructuredContent', 'noTextContent'])


In [18]:
print(md_json_list[0]['images'])

[{'name': 'img_p0_3.png', 'height': 137, 'width': 300, 'x': 15.348425687999999, 'y': 219.125947956912, 'original_width': 300, 'original_height': 137, 'ocr': [{'x': 106, 'y': 34, 'w': 116, 'h': 24, 'confidence': '0.9842973394927044', 'text': 'SINGAPORE'}, {'x': 21, 'y': 47, 'w': 64, 'h': 40, 'confidence': '0.9812295739401005', 'text': 'WS'}, {'x': 106, 'y': 54, 'w': 194, 'h': 26, 'confidence': '0.4937106920834399', 'text': 'WorkforCE SKILL8'}, {'x': 106, 'y': 75, 'w': 159, 'h': 24, 'confidence': '0.6521510474818355', 'text': 'QUALIFICATIONS'}]}, {'name': 'img_p0_4.png', 'height': 80, 'width': 350, 'x': 82.51181366399999, 'y': 311.43105327288, 'original_width': 350, 'original_height': 80, 'ocr': [{'x': 2, 'y': 6, 'w': 348, 'h': 74, 'confidence': '0.37336243117196827', 'text': 'Umfmtech'}]}, {'name': 'img_p0_5.png', 'height': 400, 'width': 400, 'x': 12.000000384, 'y': 306.91363186847997, 'original_width': 400, 'original_height': 400, 'ocr': [{'x': 86, 'y': 40, 'w': 246, 'h': 306, 'confide

In [19]:
image_dicts = parser.get_images(md_json_objs, download_path="data_images")

> Image for page 1: [{'name': 'img_p0_3.png', 'height': 137, 'width': 300, 'x': 15.348425687999999, 'y': 219.125947956912, 'original_width': 300, 'original_height': 137, 'ocr': [{'x': 106, 'y': 34, 'w': 116, 'h': 24, 'confidence': '0.9842973394927044', 'text': 'SINGAPORE'}, {'x': 21, 'y': 47, 'w': 64, 'h': 40, 'confidence': '0.9812295739401005', 'text': 'WS'}, {'x': 106, 'y': 54, 'w': 194, 'h': 26, 'confidence': '0.4937106920834399', 'text': 'WorkforCE SKILL8'}, {'x': 106, 'y': 75, 'w': 159, 'h': 24, 'confidence': '0.6521510474818355', 'text': 'QUALIFICATIONS'}]}, {'name': 'img_p0_4.png', 'height': 80, 'width': 350, 'x': 82.51181366399999, 'y': 311.43105327288, 'original_width': 350, 'original_height': 80, 'ocr': [{'x': 2, 'y': 6, 'w': 348, 'h': 74, 'confidence': '0.37336243117196827', 'text': 'Umfmtech'}]}, {'name': 'img_p0_5.png', 'height': 400, 'width': 400, 'x': 12.000000384, 'y': 306.91363186847997, 'original_width': 400, 'original_height': 400, 'ocr': [{'x': 86, 'y': 40, 'w': 246

In [20]:
from llama_index.core.schema import TextNode
from typing import Optional

In [21]:
# get pages loaded through llamaparse
import re

def get_page_number(file_name):
    match = re.search(r"-page-(\d+)\.jpg$", str(file_name))
    if match:
        return int(match.group(1))
    return 0


def _get_sorted_image_files(image_dir):
    """Get image files sorted by page."""
    raw_files = [f for f in list(Path(image_dir).iterdir()) if f.is_file()]
    sorted_files = sorted(raw_files, key=get_page_number)
    return sorted_files

In [22]:
from copy import deepcopy
from pathlib import Path


# attach image metadata to the text nodes
def get_text_nodes(json_dicts, image_dir=None):
    """Split docs into nodes, by separator."""
    nodes = []

    image_files = _get_sorted_image_files(image_dir) if image_dir is not None else None
    md_texts = [d["text"] for d in json_dicts]

    for idx, md_text in enumerate(md_texts):
        chunk_metadata = {"page_num": idx + 1}
        # Check if an image exists for the current index
        if idx < len(image_files):
            chunk_metadata["image_path"] = str(image_files[idx])
        else:
            chunk_metadata["image_path"] = None  # No image available for this page
        
        # if image_files is not None:
        #     image_file = image_files[idx]
        #     chunk_metadata["image_path"] = str(image_file)
        chunk_metadata["parsed_text_markdown"] = md_text
        node = TextNode(
            text="",
            metadata=chunk_metadata,
        )
        nodes.append(node)

    return nodes

In [23]:
# this will split into pages
text_nodes = get_text_nodes(md_json_list, image_dir="data_images")

In [24]:
print(text_nodes[10].get_content(metadata_mode="all"))

page_num: 11
image_path: data_images\af26f91b-a611-44b3-91e9-f0684926e6c1-img_p103_1.png
parsed_text_markdown:                             Final Assessment
●  Written Assessment (SAQ) - 1 hr
●  Practical Performance (PP) - 1 hr

                                                                                                            11
                        This material belongs to Tertiary Infotech Pte Ltd (UEN: 20120096W). All Rights Reserved


In [25]:
import os
from llama_index.core import (
    StorageContext,
    SummaryIndex,
    load_index_from_storage,
)

if not os.path.exists("storage_nodes_summary"):
    index = SummaryIndex(text_nodes)
    # save index to disk
    index.set_index_id("summary_index")
    index.storage_context.persist("./storage_nodes_summary")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage_nodes_summary")
    # load index
    index = load_index_from_storage(storage_context, index_id="summary_index")

### Extraction of FG Document Data

In this section, we focus on extracting structured data from the Facilitator Guide (FG) document. The process involves several key steps:

1. **Parsing the FG Document**: We utilize the `LlamaParse` tool to preprocess the content of the FG document. This involves reading the document and converting it into a structured format that can be easily processed.

2. **Loading Document Content**: The content of the FG document is loaded using the `Document` class from the `docx` module. This allows us to read the text content of the document and prepare it for further processing.

3. **Extracting Structured Data**: Using the OpenAI model, we extract relevant information from the preprocessed content. This includes details such as the course title, TSC proficiency level, learning units, topics, knowledge statements, ability statements, and assessment methods. The extracted data is validated against the `FacilitatorGuideExtraction` Pydantic model to ensure it adheres to the expected schema.

In [26]:
from pydantic import BaseModel
from openai import OpenAI
from docx import Document
from typing import Dict, List
from llama_parse import LlamaParse
import nest_asyncio
import os
import json

# API access to llama-cloud
LLAMA_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")

# Using OpenAI API for embeddings/llms
OPENAI_API_KEY = os.getenv("TERTIARY_INFOTECH_API_KEY")

# Initialize the OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

class KnowledgeStatement(BaseModel):
    id: str
    text: str


class AbilityStatement(BaseModel):
    id: str
    text: str


class Topic(BaseModel):
    name: str
    subtopics: List[str]
    tsc_knowledges: List[KnowledgeStatement]
    tsc_abilities: List[AbilityStatement]


class LearningUnit(BaseModel):
    name: str
    topics: List[Topic]
    learning_outcome: str


class AssessmentMethod(BaseModel):
    code: str
    duration: str
    
class FacilitatorGuideExtraction(BaseModel):
    course_title: str
    tsc_proficiency_level: str
    learning_units: List[LearningUnit]
    assessments: List[AssessmentMethod]  # New field for assessments


# Function to load FG document content using LlamaParse
def parse_with_llamaparse(file_path: str) -> str:
    """
    Parses the FG document using LlamaParse to preprocess its content.

    Args:
        file_path (str): Path to the document.

    Returns:
        str: Preprocessed content of the document as text.
    """

    parser = LlamaParse(
        api_key=LLAMA_API_KEY,
        result_type="markdown",
        show_progress=True,
        verbose=True,
        num_workers=8,
        fast_mode=True,
    )

    result = parser.get_json_result(file_path)
    return result

# Function to load FG document content
def load_fg_document(file_path: str) -> str:
    """
    Reads the content of a Word document.

    Args:
        file_path (str): Path to the document.

    Returns:
        str: The document's text content.
    """
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

# Function to extract structured data from FG document
def extract_fg_data(preprocessed_content: str) -> FacilitatorGuideExtraction:
    """
    Extracts relevant information from the FG document using GPT and Pydantic.

    Args:
        preprocessed_content (str): Preprocessed text content of the FG document.

    Returns:
        FacilitatorGuideExtraction: Extracted and validated data.
    """
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    """You are an expert at structured data extraction. Extract the following details from the FG Document:
                    - Course Title
                    - TSC Proficiency Level
                    - Learning Units (LUs):
                        * Name of the Learning Unit
                        * Topics in the Learning Unit:
                            - Name of the Topic
                            - Description of the Topic (bullet points or sub-topics)
                            - Full Knowledge Statements associated with the topic, including their identifiers and text (e.g., K1: Range of AI applications)
                            - Full Ability Statements associated with the topic, including their identifiers and text (e.g., A1: Analyze algorithms in the AI applications)
                        * Learning Outcome (LO) for each Learning Unit
                    - Assessment Types and Durations:
                        * Extract assessment types and their durations in the format:
                          {"code": "WA-SAQ", "duration": "1 hr"}
                          {"code": "PP", "duration": "0.5 hr"}
                          {"code": "CS", "duration": "30 mins"}
                        * Interpret abbreviations of assessment methods to their correct types (e.g., "WA-SAQ," "PP," "CS").
                        * Include total durations if mentioned.

                    Return the output in a JSON format that matches the schema provided:
                    {
                        "course_title": "string",
                        "tsc_proficiency_level": "string",
                        "learning_units": [
                            {
                                "name": "string",
                                "topics": [
                                    {
                                        "name": "string",
                                        "subtopics": ["string"],
                                        "tsc_knowledges": [
                                            {"id": "string", "text": "string"}
                                        ],
                                        "tsc_abilities": [
                                            {"id": "string", "text": "string"}
                                        ]
                                    }
                                ],
                                "learning_outcome": "string"
                            }
                        ],
                        "assessments": [
                            {"code": "string", "duration": "string"}
                        ]
                    }
                    """
                ),
            },
            {"role": "user", "content": preprocessed_content},
        ],
        response_format=FacilitatorGuideExtraction,
    )

    return completion.choices[0].message.parsed

# Main Execution
if __name__ == "__main__":
    # Path to the FG document
    file_path = r"../input/FG_TGS-2024042961_Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini_v2.docx"
    
    # Parse document content using LlamaParse
    try:
        parsed_content = parse_with_llamaparse(file_path)
        print("Preprocessed Content:")
        print(json.dumps(parsed_content, indent=4))
    except Exception as e:
        print(f"Error during LlamaParse parsing: {e}")
        exit(1)

    # Extract and validate structured data using OpenAI and Pydantic
    try:
        extracted_data = extract_fg_data(json.dumps(parsed_content))
        print("Structured Data:")
        print(extracted_data)
    except Exception as e:
        print(f"Error during OpenAI extraction: {e}")


Started parsing the file under job_id a0f4455d-9be6-4f9a-ba62-7f3ccc88d9ba
Preprocessed Content:
[
    {
        "pages": [
            {
                "page": 1,
                "text": "                                    T\n                              Facilitator Guide\n                                         For\n\nDevelop Artificial Intelligence and Large Language Model (LLM) Applications\n                                with Google Gemini\n\n                           TGS Ref No: TGS-2024042961\n\n                                   Conducted by\n                         TERTIARY INFOTECH PTE. LTD\n\n                                 UEN: 201200696W\n\n                                     Version 2.0",
                "images": [
                    {
                        "name": "img_p0_1.png",
                        "height": 500,
                        "width": 500,
                        "x": 225,
                        "y": 158.30076377952798,
                     

In [27]:
print(extracted_data)

course_title='Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini' tsc_proficiency_level='3' learning_units=[LearningUnit(name='Overview of Large Language Model (LLM)', topics=[Topic(name='Overview of Large Language Model (LLM)', subtopics=['What is Large Language Model (LLM)?', 'Opportunities LLM applications', 'Industrial use cases of LLM applications'], tsc_knowledges=[KnowledgeStatement(id='K1', text='Range of AI applications'), KnowledgeStatement(id='K6', text='Applicability of AI in the industry')], tsc_abilities=[AbilityStatement(id='A1', text='Analyse algorithms in the AI applications'), AbilityStatement(id='A3', text='Identify strengths and limitations of the AI applications')])], learning_outcome='Analyze the range of LLM applications using Generative AI (GAI) and identify their industrial use cases.'), LearningUnit(name='Multimodal Prompting with Google Gemini LLM', topics=[Topic(name='Multimodal Prompting with Google Gemini LLM', s

## Generating Practical Performance Scenario-based Questions (PP)

In this section, we focus on generating Practical Performance (PP) scenario-based questions. These questions are designed to assess learners ability to apply their knowledge and skills in realistic, practical situations. The process involves several key steps:## Generating Practical Performance Scenario-based Questions (PP)
In this section, we focus on generating Practical Performance (PP) scenario

### Scenario Generation Agent for Practical Performance Assessment

The Scenario Generation Agent is designed to create detailed, realistic scenarios specifically for practical performance assessments. By leveraging the provided Learning Outcomes, Course Title, and TSC Proficiency Level, the agent generates comprehensive scenarios that align with the educational goals and proficiency requirements. These scenarios serve as practical contexts for learners to apply their knowledge and skills, ensuring that the learning experience is both relevant and engaging.

The agent uses advanced language models to craft scenarios that are approximately 500 words in length, providing sufficient detail to cover the complexities and nuances of real-world situations. These scenarios are tailored to highlight specific organizational challenges, data points, and objectives, making them highly applicable to the learners' future professional environments.

Key features of the Scenario Generation Agent include:
- **Alignment with Learning Outcomes**: Ensures that the generated scenario directly supports the specified learning outcomes, helping learners achieve the desired educational objectives.
- **Relevance to Course Title**: Incorporates elements related to the course title, ensuring that the scenario is contextually appropriate and enhances the overall learning experience.
- **TSC Proficiency Level**: Adjusts the complexity and depth of the scenario based on the specified TSC proficiency level, catering to the learners' current skill set and knowledge base.
- **Realistic and Practical**: Focuses on creating scenarios that are realistic and practical, providing learners with opportunities to apply their skills in situations that mirror real-world challenges.
- **Detailed and Comprehensive**: Generates scenarios that are approximately 500 words long, offering a thorough exploration of the context and challenges, and providing ample material for learners to engage with.

By integrating these features, the Scenario Generation Agent ensures that learners are well-prepared to tackle real-world problems, enhancing their ability to apply theoretical knowledge in practical settings.

In [44]:
from llama_index.llms.openai import OpenAI
# System prompt tailored for content retrieval and question-answer generation
system_prompt = """\
You are an instructional design assistant tasked with generating realistic, scenario-based practical performance assessment (PPA) tasks for educational purposes. Your output must focus on hands-on application and skill demonstration in a real-world context.

Your Role:
1. Develop a realistic and concise scenario for the given Course Title, Learning Outcome (LO), and associated abilities (As). Each scenario must:
   - Be relevant to the LO and abilities while addressing specific organizational or practical challenges.
   - Provide clear context (e.g., organization type, objectives, and challenges) in 1 paragraph.
   - Be hands-on and action-oriented, directly connecting to the skills or abilities being assessed.
   - Align with the Bloom's Taxonomy level specified for the LO.

2. Highlight Practical Application: The scenario must clearly depict how learners will:
   - Apply the targeted skills.
   - Solve real-world challenges or meet organizational needs.
   - Demonstrate performance relevant to the given abilities.

3. If insufficient information is available, create a general but realistic scenario tied to the broader course theme, ensuring it remains educationally valuable.

Output Format:
[Provide a single paragraph description of the situation, including organization type, objectives, challenges, and the specific task learners must complete. The task should be action-oriented and tied directly to the LO and abilities.]

Example Scenario 1:
TechRobotics, a company specializing in developing intelligent robots for industrial automation, is conducting a training program to ensure their engineers are proficient in using the Robot Operating System (ROS). The training covers essential skills such as installing ROS, creating simulations, diagnosing communication issues, and controlling robots with ROS packages. As part of the training team, you are responsible for overseeing the use of ROS, diagnosing issues, interpreting specifications, applying corrective actions, and incorporating feedback into operational procedures.

Example Scenario 2:
GreenEarth Innovations, a sustainable lifestyle brand, is launching a new line of eco-friendly skincare products. The company aims to leverage XiaoHongShu (Little Red Book) to reach a younger, environmentally-conscious audience. The marketing team is tasked with creating an engaging social e-commerce campaign that aligns with the company’s strategic goals, monitors campaign performance, analyzes sales and feedback, identifies trends, and engages with customer feedback on social media and forums.

Restrictions:
- Focus exclusively on the given LO and abilities.
- Avoid using unrelated topics or assumptions.
- Do not invent abilities or knowledge outside the provided scope.
"""

scenario_llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, system_prompt=system_prompt)

In [45]:
scenario_query_engine = index.as_query_engine(
    similarity_top_k=10,
    llm=scenario_llm,
    response_mode="compact",
)

In [46]:
# Generate a detailed scenario for the case study
def generate_pp_scenario(data: FacilitatorGuideExtraction) -> str:
    """
    Generates a concise, realistic scenario for the practical performance.
    Args:
        course_title (str): The title of the course.
        learning_outcomes (List[str]): A list of learning outcomes.

    Returns:
        str: A concise scenario for the case study.
    """
    
    # Retrieve the course title and bloom taxonomy level
    course_title = data.course_title
    bloom_taxonomy_level = data.tsc_proficiency_level

    # Extract the learning outcomes as a list of strings
    learning_outcomes = [lu.learning_outcome for lu in data.learning_units]
    abilities = [ability.text for lu in data.learning_units for topic in lu.topics for ability in topic.tsc_abilities]
    
    outcomes_text = "\n".join([f"- {lo}" for lo in learning_outcomes])
    abilities_text = "\n".join([f"- {ability}" for ability in abilities])

    prompt = (
        f"You are tasked with designing a realistic practical performance assessment scenario for the course '{course_title}'.\n\n"
        f"The scenario should align with the following:\n\n"
        f"Learning Outcomes:\n{outcomes_text}\n\n"
        f"Abilities:\n{abilities_text}\n\n"
        f"Bloom's Taxonomy Level:\n{bloom_taxonomy_level}\n\n"
        "The scenario should describe a company or organization facing practical challenges and provide context for the learners to apply their skills.\n"
        "Ensure the scenario is concise (1 paragraph), realistic, and action-oriented, focusing on the summary of tasks learners must perform without requiring extensive deliverables."
    )
    response = scenario_query_engine.query(prompt)
    return response.response.strip()

# Generate the shared scenario
scenario = generate_pp_scenario(extracted_data)

In [47]:
from IPython.display import display, Markdown
display(Markdown(scenario))

In a rapidly evolving tech landscape, InnovateAI, a mid-sized company specializing in AI-driven solutions for the healthcare sector, is facing challenges in optimizing its patient management system. The company has recently integrated Google Gemini's Generative AI capabilities to enhance its application for real-time patient data analysis and decision support. As part of a project team, you are tasked with analyzing the current LLM applications to identify their strengths and limitations, establishing the design of algorithms that improve efficiency in processing patient data, and assessing the feasibility of these AI applications in streamlining engineering processes. Additionally, you will evaluate the performance effectiveness of the Retrieval Augmented Generation (RAG) model implemented in the system, ensuring that the solutions align with industry standards and enhance overall operational efficiency. Your role will involve hands-on analysis, algorithm design, and performance evaluation to provide actionable insights that drive improvements in the company's healthcare solutions.

### Question and Answer Generation Agent

The Question and Answer Generation Agent is designed to facilitate the creation of educational content by generating scenario-based questions and answers. This agent operates by taking in a specified Learning Outcome (LO) and its associated abilities, retrieving relevant content, and then generating questions and answers that align with the given scenario and Bloom's Taxonomy level.

#### Key Features:
1. **Input Handling**:
    - **Learning Outcome (LO)**: The specific educational goal that the learners are expected to achieve.
    - **Associated Abilities**: The skills or competencies that are linked to the Learning Outcome.

2. **Content Retrieval**:
    - The agent retrieves content that is relevant to the specified Learning Outcome and its associated abilities. This ensures that the generated questions and answers are contextually appropriate and aligned with the educational objectives.

3. **Scenario Alignment**:
    - The agent generates a realistic and practical scenario that aligns with the Learning Outcome. This scenario provides a context for the questions and answers, making them more engaging and applicable to real-world situations.

4. **Question and Answer Generation**:
    - Based on the retrieved content and the generated scenario, the agent formulates questions that require learners to demonstrate their understanding and mastery of the Learning Outcome and associated abilities.
    - The answers are crafted to align with Bloom's Taxonomy level, ensuring that they meet the desired cognitive complexity and educational standards.

5. **Output**:
    - The final output includes the scenario, the question, and the answer, all structured in a clear and concise format. This output can be used directly in educational assessments or as part of instructional materials.

By integrating these features, the Question and Answer Generation Agent ensures that the generated educational content is both relevant and effective in achieving the specified learning objectives.

## Generating Case Study Scenario-based Questions (CS Scenario-based Questions)

In this section, we generate the Case Study assessment questions and answers. Each question is linked to a specific learning outcome and includes a question, an answer, and its associated ability statements, which are structured in a JSON format. The results are then saved to a file for further use.

### Retrieve the Learning Outcomes and its associated Ability Statements

In [34]:
# Print the details
print("Course Title:", extracted_data.course_title)
print("Bloom's Taxonomy Level:", extracted_data.tsc_proficiency_level)
print("\nLearning Outcomes with Abilities:\n")

for learning_unit in extracted_data.learning_units:
    print(f"Learning Outcome: {learning_unit.learning_outcome}")
    print("Associated Ability Statements:")
    
    # Loop through topics to gather abilities
    for topic in learning_unit.topics:
        for ability in topic.tsc_abilities:
            print(f"  - [{ability.id}] {ability.text}")
    
    print("-" * 20)



Course Title: Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini
Bloom's Taxonomy Level: 3

Learning Outcomes with Abilities:

Learning Outcome: Analyze the range of LLM applications using Generative AI (GAI) and identify their industrial use cases.
Associated Ability Statements:
  - [A1] Analyse algorithms in the AI applications
  - [A3] Identify strengths and limitations of the AI applications
--------------------
Learning Outcome: Establish Google Gemini GAI designs and assess improvements on engineering processes.
Associated Ability Statements:
  - [A2] Establish the correlation between design of algorithms and efficiency
  - [A6] Assess improvements on the engineering and maintenance processes
--------------------
Learning Outcome: Develop LLM applications and assess its feasibility.
Associated Ability Statements:
  - [A5] Assess feasibility of AI applications to the engineering processes
--------------------
Learning Outcome: Evaluate t

### Scenario Generation Agent for Case Study
The Scenario Generation Agent is designed to create detailed, realistic scenarios based on specific educational inputs. By leveraging the provided Learning Outcomes, Course Title, and TSC Proficiency Level, the agent generates a comprehensive scenario that aligns with the educational goals and proficiency requirements. This scenario serves as a practical context for learners to apply their knowledge and skills, ensuring that the learning experience is both relevant and engaging.

The agent uses advanced language models to craft scenarios that are approximately 500 words in length, providing sufficient detail to cover the complexities and nuances of real-world situations. These scenarios are tailored to highlight specific organizational challenges, data points, and objectives, making them highly applicable to the learners' future professional environments.

Key features of the Scenario Generation Agent include:
- **Alignment with Learning Outcomes**: Ensures that the generated scenario directly supports the specified learning outcomes, helping learners achieve the desired educational objectives.
- **Relevance to Course Title**: Incorporates elements related to the course title, ensuring that the scenario is contextually appropriate and enhances the overall learning experience.
- **TSC Proficiency Level**: Adjusts the complexity and depth of the scenario based on the specified TSC proficiency level, catering to the learners' current skill set and knowledge base.
- **Realistic and Practical**: Focuses on creating scenarios that are realistic and practical, providing learners with opportunities to apply their skills in situations that mirror real-world challenges.
- **Detailed and Comprehensive**: Generates scenarios that are 1-2 paragraphs long, offering a thorough exploration of the context and challenges, and providing ample material for learners to engage with.

By integrating these features, the Scenario Generation Agent ensures that learners are well-prepared to tackle real-world problems, enhancing their ability to apply theoretical knowledge in practical settings.

In [84]:
from llama_index.llms.openai import OpenAI
# System prompt tailored for content retrieval and question-answer generation
system_prompt = """\
You are an instructional design assistant tasked with generating concise, realistic, and practical scenario-based question-answer pairs for educational purposes.

Your role:
1. **Generate a real-world scenario** for the given Course Title and Learning Outcome (LO). The scenario must:
   - Be concise (1-2 paragraphs) while clearly describing the organizational challenges or context.
   - Align directly with the Learning Outcome and be applicable to the associated abilities.
   - Highlight specific organizational data, challenges, and objectives to ensure relevance and practicality.

2. Use only the information relevant to the specified Learning Unit, Learning Outcome, and its abilities. Do not include information from unrelated topics.

3. Ensure that:
   - Each scenario and question-answer pair is realistic, aligned to Bloom's Taxonomy level for the LO, and practically applicable.
   - If no relevant content exists, create a general scenario that remains educationally valuable and tied to the broader course theme.

**Output Format:**
- Tase study scenario have to be at least 500 words long.
- You will output your response in the following format. For example:
TechFusion, a leading software solutions provider, has been approached by a global bank to develop a new mobile banking application. The bank wants to offer its customers a seamless, secure, and intuitive banking experience on their smartphones. Given the competitive landscape, the bank emphasizes the need for rapid delivery without compromising on quality. TechFusion has recently adopted Agile methodologies with Scrum and DevOps practices and sees this project as an opportunity to showcase its capabilities in these areas.

**Restrictions:**
- Do not include content from other topics or unrelated slides.
- Do not invent abilities or knowledge outside the scope of the LO and its associated abilities.
"""

scenario_llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("PERSONAL_OAI_API_KEY"), system_prompt=system_prompt)

In [85]:
scenario_query_engine = index.as_query_engine(
    similarity_top_k=10,
    llm=scenario_llm,
    response_mode="compact",
)

In [86]:
# Generate a detailed scenario for the case study
def generate_case_study_scenario(data: FacilitatorGuideExtraction) -> str:
    """
    Generates a concise, realistic scenario for the case study.
    Args:
        course_title (str): The title of the course.
        learning_outcomes (List[str]): A list of learning outcomes.

    Returns:
        str: A concise scenario for the case study.
    """
    
    # Retrieve the course title and bloom taxonomy level
    course_title = data.course_title
    bloom_taxonomy_level = data.tsc_proficiency_level

    # Extract the learning outcomes as a list of strings
    learning_outcomes = [lu.learning_outcome for lu in data.learning_units]
    abilities = [ability.text for lu in data.learning_units for topic in lu.topics for ability in topic.tsc_abilities]
    
    outcomes_text = "\n".join([f"- {lo}" for lo in learning_outcomes])
    abilities_text = "\n".join([f"- {ability}" for ability in abilities])

    prompt = (
        f"You are tasked with designing a concise, realistic case study scenario for the course '{course_title}'.\n\n"
        f"The scenario should align with the following:\n\n"
        f"Learning Outcomes:\n{outcomes_text}\n\n"
        f"Abilities:\n{abilities_text}\n\n"
        f"Bloom's Taxonomy Level:\n{bloom_taxonomy_level}\n\n"
        "The scenario should describe a company or organization facing challenges related to communication, collaboration, or customer satisfaction. "
        "Ensure the scenario is realistic and practical, and keep it to 1-2 paragraphs without markdown elements or formatting."
    )
    response = scenario_query_engine.query(prompt)
    return response.response.strip()

# Generate the shared scenario
scenario = generate_case_study_scenario(extracted_data)

Retrying llama_index.llms.openai.base.OpenAI._chat in 0.9756561110559212 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.
Retrying llama_index.llms.openai.base.OpenAI._chat in 0.006215702531275014 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [42]:
print(scenario)

TechSolutions Inc., a mid-sized technology consulting firm, has been experiencing significant challenges in managing client communications and project updates. With a growing portfolio of clients, the firm struggles to maintain timely and effective communication, leading to misunderstandings and delays in project deliverables. To address these issues, the management team has decided to explore the implementation of a Large Language Model (LLM) application using Google Gemini. They aim to analyze various LLM applications that can automate client interactions, streamline project updates, and enhance overall customer satisfaction. The team is particularly interested in assessing the feasibility of integrating this technology into their existing workflows and evaluating its potential to improve engineering processes and client engagement.

As part of this initiative, the team will analyze the algorithms behind the proposed LLM applications, identify their strengths and limitations, and est

In [41]:
from IPython.display import display, Markdown
display(Markdown(scenario))

TechSolutions Inc., a mid-sized technology consulting firm, has been experiencing significant challenges in managing client communications and project updates. With a growing portfolio of clients, the firm struggles to maintain timely and effective communication, leading to misunderstandings and delays in project deliverables. To address these issues, the management team has decided to explore the implementation of a Large Language Model (LLM) application using Google Gemini. They aim to analyze various LLM applications that can automate client interactions, streamline project updates, and enhance overall customer satisfaction. The team is particularly interested in assessing the feasibility of integrating this technology into their existing workflows and evaluating its potential to improve engineering processes and client engagement.

As part of this initiative, the team will analyze the algorithms behind the proposed LLM applications, identify their strengths and limitations, and establish correlations between the design of these algorithms and their efficiency in real-world scenarios. They will also evaluate the performance effectiveness of a Retrieval Augmented Generation (RAG) approach to ensure that the LLM can provide accurate and contextually relevant responses to client inquiries. By the end of the project, TechSolutions Inc. aims to enhance its communication strategies, improve client satisfaction, and ultimately drive better project outcomes.

### Retrieving Content Aligning to Learning Outcomes from the Vector Store

In this section, we focus on retrieving content that aligns with specific learning outcomes from the vector store. The process involves querying the vector store using the learning outcomes and associated abilities to extract relevant content. This ensures that the retrieved content is contextually appropriate and supports the educational objectives.

#### Steps Involved:

1. **Define the Learning Outcomes and Abilities**:
    - Identify the learning outcomes and their associated abilities from the extracted data.

2. **Formulate the Retrieval Prompt**:
    - Create a prompt that specifies the learning outcome and associated abilities.
    - The prompt should clearly state the requirement to retrieve content that aligns with the specified learning outcome.

3. **Query the Vector Store**:
    - Use the formulated prompt to query the vector store.
    - The query engine will retrieve the most relevant content based on the learning outcome and abilities.

4. **Process and Display the Retrieved Content**:
    - Process the retrieved content to ensure it aligns with the learning outcome.
    - Display the retrieved content in a structured format for further use.

In [48]:
from llama_index.llms.openai import OpenAI
from pydantic.v1 import BaseModel, Field
from typing import List
from IPython.display import display, Markdown, Image

# System prompt tailored for content retrieval and question-answer generation
system_prompt = """\
You are a content retrieval assistant. Your role is to retrieve topic content that aligns strictly with the specified Learning Outcome (LO) and its associated abilities.

Your role:
1. Restrict your retrieval strictly to the specified topic provided in the query.
2. Retrieve content from the topic that directly aligns with the provided Learning Outcome (LO) and its abilities.
3. If no specific content directly aligns with the Learning Outcome or abilities, provide a general summary of the topic instead.
4. Include any example/usecase code or equations relevant to the topic or subtopics.
5. Prioritize retrieving content that are practical.
6. Identify and extract the exact inline segments from the provided documents that directly correspond to the content used to generate the summary. The extracted segments must be verbatim snippets from the documents, ensuring a word-for-word match with the text in the provided documents.

Ensure that:
- (Important) Each retrieved segment is an exact match to a part of the document and is fully contained within the document text.
- The relevance of each segment to the Learning Outcome or abilities is clear and directly supports the summary provided.
- (Important) If you didn't use the specific document or topic, do not mention it.
- If no relevant information is found for the Learning Outcome, clearly state this and provide a general topic summary instead.

Restrictions:
- Do not include content from other topics or slides outside the specified topic.
- Each retrieved segment must explicitly belong to the given topic.
- Avoid including assumptions or content outside the scope of the Learning Outcome and abilities.

You must always provide:
1. The retrieved content aligned with the Learning Outcome and abilities.
2. A list of verbatim extracted segments that directly support the retrieved content, each labeled with the topic and document it belongs to.
"""


lo_retriever_llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, system_prompt=system_prompt)
qa_generation_query_engine = index.as_query_engine(
    similarity_top_k=10,
    llm=lo_retriever_llm,
    response_mode="compact",
)

In [49]:
# Define the content retrieval prompt
retrieval_prompt = (
    f"Retrieve the most relevant inline segments aligned to Learning Outcome: Evaluate the performance effectiveness of Retrieval Augmented Generation (RAG)\n"
    f"Associated Abilities:\n"
    f"A4: Evaluate various AI applications to compare strengths and limitations of the AI applications\n"
    f"from the given Topics: Topic 4 Implementing Retrieval Augmented Generation (RAG)"
)
response = qa_generation_query_engine.query(retrieval_prompt)
display(Markdown(response.response))

To evaluate the performance effectiveness of Retrieval Augmented Generation (RAG), the following relevant content has been extracted:

1. **Overview of RAG**:
   - RAG combines the strengths of language models with external knowledge sources to enhance the quality of generated responses. It utilizes a retrieval mechanism to fetch relevant documents that inform the generation process.

2. **Use Cases of RAG**:
   - RAG can be applied in various scenarios such as:
     - Question Answering
     - Information Retrieval
     - Content Recommendation
     - Data Enrichment
     - Knowledge Base Construction
     - Academic Research
     - Legal Research
     - Language Translation
     - Customer Support
     - Educational Tools

3. **RAG Process**:
   - The RAG process involves several steps:
     - **Document Loading**: Accessing and converting data from various sources.
     - **Document Splitting**: Breaking down documents into manageable chunks for processing.
     - **Storage**: Storing the processed documents in a vector database.
     - **Retrieval**: Fetching relevant document splits based on user queries.
     - **Output**: Generating answers based on the retrieved documents.

4. **Example of RAG Implementation**:
   - The implementation of RAG can be demonstrated through the following code snippet:
     ```python
     from langchain.chains import RetrievalQA
     from langchain.document_loaders import CSVLoader
     from langchain.vectorstores import Chroma

     # Load documents
     loader = CSVLoader(file_path='path/to/documents.csv')
     documents = loader.load()

     # Create a vector store
     vectordb = Chroma.from_documents(documents)

     # Set up the RAG chain
     qa_chain = RetrievalQA.from_chain_type(
         llm=ChatOpenAI(temperature=0),
         retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
         chain_type="stuff"
     )

     # Query the RAG system
     result = qa_chain({"query": "What is the impact of RAG on AI applications?"})
     print(result)
     ```

### Extracted Segments:
- **Document Loading**: "Loaders deal with the specifics of accessing and converting data."
- **Use Cases of RAG**: "Question Answering, Information Retrieval, Content Recommendation, Data Enrichment, Knowledge Base Construction, Academic Research, Legal Research, Language Translation, Customer Support, Educational Tools."
- **RAG Process**: "The RAG process involves several steps: Document Loading, Document Splitting, Storage, Retrieval, Output."
- **Example of RAG Implementation**: "from langchain.chains import RetrievalQA... result = qa_chain({'query': 'What is the impact of RAG on AI applications?'})"

These segments provide a comprehensive understanding of how to evaluate the performance effectiveness of RAG, aligning with the specified Learning Outcome and associated abilities.

In [50]:
def retrieve_content_for_learning_outcomes(extracted_data):
    """
    Retrieves content related to the learning outcomes and abilities from the provided data.

    Args:
        extracted_data (FacilitatorGuideExtraction): The extracted data instance containing course details.

    Returns:
        List[Dict]: A list of dictionaries containing retrieved content and associated abilities.
    """
    retrieved_content = []

    for learning_unit in extracted_data.learning_units:
        learning_outcome = learning_unit.learning_outcome
        associated_abilities = []
        ability_ids = []
        for topic in learning_unit.topics:
            associated_abilities.extend(topic.tsc_abilities)
            ability_ids.extend([ability.id for ability in topic.tsc_abilities])

        # Define the content retrieval prompt
        retrieval_prompt = (
            f"Retrieve the most relevant inline segments aligned to Learning Outcome: {learning_outcome}\n"
            f"Associated Abilities:\n"
            + "\n".join([f"- [{ability.id}] {ability.text}" for ability in associated_abilities])
            + f"\nFrom the given Topics: {', '.join([topic.name for topic in learning_unit.topics])}"
        )


        response = qa_generation_query_engine.query(retrieval_prompt)
        retrieved_content.append({
            "learning_outcome": learning_outcome,
            "abilities": ability_ids,
            "retrieved_content": response.response
        })
    
    return retrieved_content

In [51]:
retrieved_content = retrieve_content_for_learning_outcomes(extracted_data)

In [52]:
# Display the content in Markdown format
for content in retrieved_content:
    markdown_output = (
        f"### Learning Outcome: {content['learning_outcome']}\n"
        f"### Associated Abilities:\n"
        + "".join([f"- {ability_id}\n" for ability_id in content['abilities']])
        + f"### Retrieved Content:\n{content['retrieved_content']}\n"
    )

    display(Markdown(markdown_output))

### Learning Outcome: Analyze the range of LLM applications using Generative AI (GAI) and identify their industrial use cases.
### Associated Abilities:
- A1
- A3
### Retrieved Content:
To analyze the range of LLM applications using Generative AI (GAI) and identify their industrial use cases, the following relevant content has been extracted:

### Retrieved Content:
1. **Opportunities of LLM Applications**:
   - Content Creation and Assistance
   - Customer Support and Chatbots
   - Language Translation and Localization
   - Educational Tools
   - Business Intelligence and Analytics
   - Accessibility for Disabled Persons
   - Coding and Development
   - Legal and Compliance Assistance
   - Healthcare Support
   - Art and Design Inspiration
   - Enhanced Search Engines
   - Crisis Management and Response

2. **Use Cases of GAI in Various Industries**:
   - **Finance**:
     - Automated Financial Reporting
     - Risk Assessment and Analysis
     - Algorithmic Trading Strategies
     - Fraud Detection and Prevention
     - Customer Service Chatbots
     - Personalized Financial Advice
     - Credit Scoring and Lending Analysis
     - Regulatory Compliance and Monitoring
     - Market Sentiment Analysis
     - Contract Review and Analysis
   - **Marketing**:
     - Content Creation
     - Market Research Analysis
     - Personalized Marketing
     - Email Marketing Automation
     - Social Media Management
     - SEO Optimization
     - Customer Sentiment Analysis
     - Chatbots for Customer Engagement
     - Predictive Analytics
     - Brand Strategy Development
   - **Education**:
     - Personalized Learning Assistance
     - Grading and Feedback Automation
     - Curriculum Development and Content Creation
     - Language Learning and Translation
     - Educational Chatbots
     - Research and Writing Assistance
     - Interactive Learning Environments
     - Special Education Support
     - Teacher Training and Professional Development
     - Student Engagement and Motivation Tools
   - **Healthcare**:
     - Drug discovery and development
     - Personalized medicine and treatment optimization
     - Medical imaging and diagnosis
     - Disease prediction and prevention
     - Biomarker development
     - Synthetic data generation for research
     - Robotic surgery
     - Virtual health assistants
     - Epidemiological research
     - Enhanced patient education and engagement
     - Genomic sequencing analysis
     - Automated clinical documentation

### Extracted Segments:
- **Opportunities of LLM Applications**:
  - "Content Creation and Assistance"
  - "Customer Support and Chatbots"
  - "Language Translation and Localization"
  - "Educational Tools"
  - "Business Intelligence and Analytics"
  - "Accessibility for Disabled Persons"
  - "Coding and Development"
  - "Legal and Compliance Assistance"
  - "Healthcare Support"
  - "Art and Design Inspiration"
  - "Enhanced Search Engines"
  - "Crisis Management and Response"

- **Use Cases of GAI in Finance**:
  - "Automated Financial Reporting"
  - "Risk Assessment and Analysis"
  - "Algorithmic Trading Strategies"
  - "Fraud Detection and Prevention"
  - "Customer Service Chatbots"
  - "Personalized Financial Advice"
  - "Credit Scoring and Lending Analysis"
  - "Regulatory Compliance and Monitoring"
  - "Market Sentiment Analysis"
  - "Contract Review and Analysis"

- **Use Cases of GAI in Marketing**:
  - "Content Creation"
  - "Market Research Analysis"
  - "Personalized Marketing"
  - "Email Marketing Automation"
  - "Social Media Management"
  - "SEO Optimization"
  - "Customer Sentiment Analysis"
  - "Chatbots for Customer Engagement"
  - "Predictive Analytics"
  - "Brand Strategy Development"

- **Use Cases of GAI in Education**:
  - "Personalized Learning Assistance"
  - "Grading and Feedback Automation"
  - "Curriculum Development and Content Creation"
  - "Language Learning and Translation"
  - "Educational Chatbots"
  - "Research and Writing Assistance"
  - "Interactive Learning Environments"
  - "Special Education Support"
  - "Teacher Training and Professional Development"
  - "Student Engagement and Motivation Tools"

- **Use Cases of GAI in Healthcare**:
  - "Drug discovery and development"
  - "Personalized medicine and treatment optimization"
  - "Medical imaging and diagnosis"
  - "Disease prediction and prevention"
  - "Biomarker development"
  - "Synthetic data generation for research"
  - "Robotic surgery"
  - "Virtual health assistants"
  - "Epidemiological research"
  - "Enhanced patient education and engagement"
  - "Genomic sequencing analysis"
  - "Automated clinical documentation"

These segments provide a comprehensive overview of the applications and use cases of LLMs in various industries, aligning with the specified Learning Outcome and associated abilities.


### Learning Outcome: Establish Google Gemini GAI designs and assess improvements on engineering processes.
### Associated Abilities:
- A2
- A6
### Retrieved Content:
The content relevant to the Learning Outcome of establishing Google Gemini GAI designs and assessing improvements on engineering processes, particularly aligned with abilities A2 and A6, includes practical insights into the design and implementation of multimodal prompting with Google Gemini. 

### Retrieved Content:
1. **Multimodal Prompting with Google Gemini**:
   - This section discusses how to utilize Google Gemini for multimodal prompting, which involves integrating various types of data inputs (text, images, etc.) to enhance the efficiency and effectiveness of AI applications. The design of algorithms in this context is crucial for achieving optimal performance across different tasks.

2. **Implementation of Google AI Studio**:
   - The Google AI Studio allows for prototyping and testing designs, which can lead to improvements in engineering processes by providing a platform for experimentation and iteration. This aligns with assessing improvements in engineering processes through practical application.

3. **Example Code for API Usage**:
   - The following example demonstrates how to set up and use the Google Gemini API for generating content, which is a practical application of the design principles discussed:
   ```python
   import google.generativeai as genai
   genai.configure(api_key='YOUR_API_KEY')
   response = genai.GenerativeModel("gemini-1.5-flash").generate_content("Write a story about magic backpack.")
   print(response.text)
   ```

### Extracted Segments:
- **Topic**: Multimodal Prompting with Google Gemini LLM
  - "This section discusses how to utilize Google Gemini for multimodal prompting, which involves integrating various types of data inputs (text, images, etc.) to enhance the efficiency and effectiveness of AI applications."
  
- **Topic**: Multimodal Prompting with Google Gemini LLM
  - "The Google AI Studio allows for prototyping and testing designs, which can lead to improvements in engineering processes by providing a platform for experimentation and iteration."

- **Topic**: Multimodal Prompting with Google Gemini LLM
  - "Example code for API usage demonstrates how to set up and use the Google Gemini API for generating content."

These segments illustrate the correlation between the design of algorithms and their efficiency, as well as the assessment of improvements in engineering processes through practical applications of Google Gemini.


### Learning Outcome: Develop LLM applications and assess its feasibility.
### Associated Abilities:
- A5
### Retrieved Content:
To align with the Learning Outcome of developing LLM applications and assessing their feasibility, the following relevant content has been extracted from the topic "Building LLM Applications with Google Gemini LLM":

### Retrieved Content:
1. **Overview of LangChain Components**:
   - LangChain is a framework for developing applications powered by language models. It enables applications that are context-aware and can reason based on provided context.

2. **LangChain Libraries**:
   - The Python and JavaScript libraries facilitate the development of LLM applications, allowing for easy integration and deployment.

3. **Assessing Feasibility**:
   - The framework allows developers to assess the feasibility of integrating LLMs into existing engineering processes by providing tools for context awareness and reasoning capabilities.

4. **Example Code for Building LLM Applications**:
   ```python
   from langchain import ChatGoogleGenerativeAI
   llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-001")
   response = llm.invoke("What is the best way to implement an LLM in a customer service application?")
   print(response.content)
   ```

### Extracted Segments:
- **LangChain Overview**: "LangChain is a framework for developing applications powered by language models. It enables applications that: Are context-aware: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.) Reason: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)"
- **LangChain Libraries**: "LangChain Libraries: The Python and JavaScript libraries."
- **Assessing Feasibility**: "The framework allows developers to assess the feasibility of integrating LLMs into existing engineering processes by providing tools for context awareness and reasoning capabilities."
- **Example Code**: "from langchain import ChatGoogleGenerativeAI llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash-001') response = llm.invoke('What is the best way to implement an LLM in a customer service application?') print(response.content)"

These segments provide a clear understanding of how to develop LLM applications using the LangChain framework and assess their feasibility in engineering processes.


### Learning Outcome: Evaluate the performance effectiveness of Retrieval Augmented Generation (RAG).
### Associated Abilities:
- A4
### Retrieved Content:
To evaluate the performance effectiveness of Retrieval Augmented Generation (RAG), the following relevant content has been extracted:

### Summary of Relevant Content:
Retrieval Augmented Generation (RAG) combines the strengths of language models with external knowledge sources to enhance the quality of generated responses. It involves several key processes: document loading, splitting, storage, retrieval, and output generation. The effectiveness of RAG can be assessed through its ability to provide accurate and contextually relevant answers based on the retrieved documents. This method is particularly useful in applications such as question answering, information retrieval, and content recommendation.

### Example Code Snippets:
1. **Document Loading and Retrieval**:
   ```python
   from langchain.document_loaders import CSVLoader
   loader = CSVLoader(file_path='content/OutdoorClothingCatalog.csv')
   documents = loader.load()
   vectordb = Chroma.from_documents(documents=documents, embedding=embedding)
   ```

2. **RetrievalQA RAG**:
   ```python
   from langchain.chains import RetrievalQA
   qa_chain = RetrievalQA.from_chain_type(
       llm=llm,
       retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
       chain_type="stuff",
       return_source_documents=True
   )
   result = qa_chain({"query": "Please suggest a shirt with sunblocking"})
   ```

3. **Conversational Retrieval Chain**:
   ```python
   from langchain.chains import ConversationalRetrievalChain
   memory = ConversationBufferMemory(memory_key="chat_history")
   qa_chain = ConversationalRetrievalChain.from_llm(
       ChatOpenAI(temperature=0),
       vectorstore.as_retriever(),
       memory=memory
   )
   ```

### Extracted Segments:
1. **RAG Overview**:
   - "Retrieval Augmented Generation (RAG) combines the strengths of language models with external knowledge sources to enhance the quality of generated responses."
   
2. **Document Loading**:
   - "Loaders deal with the specifics of accessing and converting data... Returns a list of document objects."

3. **Use Cases of RAG**:
   - "Use Cases of RAG: Question Answering, Information Retrieval, Content Recommendation, Data Enrichment, Knowledge Base Construction, Academic Research, Legal Research, Language Translation, Customer Support, Educational Tools."

4. **Retrieval Process**:
   - "Relevant documents are fetched from the vector store based on the user's query, which are then used to generate a more informed response."

These segments directly support the evaluation of RAG's performance effectiveness by outlining its processes, applications, and the importance of document retrieval in generating accurate responses.


In [78]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.cache import Cache

llm_config={
        "config_list": [
            {
                'model': "gpt-4o-mini",
                'api_key': OPENAI_API_KEY,
            },
        ],
        "timeout": 120,
    }

user_proxy_agent = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: msg.get("content", "") and "TERMINATE" in msg["content"],
    code_execution_config={"work_dir": "output", "use_docker": False}
)

# Autogen setup
qa_generation_agent = AssistantAgent(
    name="Question Answer Generator",
    system_message=f"""
    You are an expert educator in '{extracted_data.course_title}'. You will create scenario-based practical performance assessment (PPA) question-answer pairs based on course data.
    The data will include:
    - A scenario
    - Retrieved content aligned with learning outcomes and abilities

    ### Instructions:
    1. Use the provided scenario and retrieved content to generate one question-and-answer pair per learning outcome.
    2. Each question must:
        - Be aligned with the scenario context and practical skills relevant to the abilities.
        - End with the statement: "Take snapshots of your ... at each step and paste them below."
    3. Each answer must:
        - Begin with: "The snapshot should include: ".
        - Specify only practical steps that test the learner's skills (e.g., commands, setups, or tool usage).
        - Avoid any form of writing, documenting, or analytical description beyond the practical steps.
    4. Ensure all keys and values are double-quoted in the JSON output.
    5. Return the output in JSON format with the following structure:
        ```json
        {{
            "course_title": "<course_title_here>",
            "duration": "<assessment_duration_here>",
            "scenario": "<scenario_here>",
            "questions": [
                {{
                    "question_statement": "<question_text>",
                    "answer": "<answer_text>",
                    "ability_id": ["<list_of_ability_ids>"]
                }},
                ...
            ]
        }}
        ```
    """,
    llm_config=llm_config
)

assessment_duration = ""
for assessment in extracted_data.assessments:
    if "PP" in assessment.code:
        assessment_duration = assessment.duration

with Cache.disk() as cache:
    chat_result = user_proxy_agent.initiate_chat(
        qa_generation_agent,
        message=f"""
        Please generate practical performance assessment questions using the following course title: '{extracted_data.course_title}', 
        assessment duration: '{assessment_duration}', scenario: '{scenario}', and topic contents: {retrieved_content}.
        Phrase your question in alignment with Bloom's Taxonomy Level: {extracted_data.tsc_proficiency_level}.
        Example Bloom's Taxonomy Levels:
            - Level 1: Remembering
            - Level 2: Understanding
            - Level 3: Applying
            - Level 4: Analyzing
            - Level 5: Evaluating
            - Level 6: Creating
        Ensure the question ends with "Take snapshots of your commands at each step and paste them below."
        Ensure the answer begins with "The snapshot should include: " and specifies only practical steps to test hands-on skills without any writing or documenting.
        RETURN 'TERMINATE' once the generation is done.
        """,
        summary_method="reflection_with_llm",
        cache=cache,
    )

[autogen.oai.client: 12-12 15:05:37] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
user_proxy (to Question Answer Generator):


        Please generate practical performance assessment questions using the following course title: 'Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini', 
        assessment duration: '1 hr', scenario: 'In a rapidly evolving tech landscape, InnovateAI, a mid-sized company specializing in AI-driven solutions for the healthcare sector, is facing challenges in optimizing its patient management system. The company has recently integrated Google Gemini's Generative AI capabilities to enhance its application for real-time patient data analysis and decision support. As part of a project team, you are tasked with analyzing the current LLM applications to identify their strengths and limitations, establishing the design of algorithms that improve efficiency in

In [79]:
import re
import json

try:
    # Extract the content from the last message
    last_message_content = chat_result.chat_history[-1].get("content", "")
    if not last_message_content:
        print("No content found in the agent's last message.")

    last_message_content = last_message_content.strip()

    # Extract JSON context from the message
    json_pattern = re.compile(r'```json\s*(\{.*?\})\s*```', re.DOTALL)
    json_match = json_pattern.search(last_message_content)
    if json_match:
        json_str = json_match.group(1)
        context = json.loads(json_str)  # Parse the extracted JSON
        print(f"CONTEXT JSON MAPPING: \n\n{context}")
    else:
        print("No JSON context found in the agent's last message.")

except json.JSONDecodeError as e:
    print(f"Error parsing context JSON: {e}")



CONTEXT JSON MAPPING: 

{'course_title': 'Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini', 'duration': '1 hr', 'scenario': "In a rapidly evolving tech landscape, InnovateAI, a mid-sized company specializing in AI-driven solutions for the healthcare sector, is facing challenges in optimizing its patient management system. The company has recently integrated Google Gemini's Generative AI capabilities to enhance its application for real-time patient data analysis and decision support. As part of a project team, you are tasked with analyzing the current LLM applications to identify their strengths and limitations, establishing the design of algorithms that improve efficiency in processing patient data, and assessing the feasibility of these AI applications in streamlining engineering processes. Additionally, you will evaluate the performance effectiveness of the Retrieval Augmented Generation (RAG) model implemented in the system, ensuring th

In [80]:
pretty_json = json.dumps(context, indent=4, ensure_ascii=False)
print(pretty_json)

{
    "course_title": "Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini",
    "duration": "1 hr",
    "scenario": "In a rapidly evolving tech landscape, InnovateAI, a mid-sized company specializing in AI-driven solutions for the healthcare sector, is facing challenges in optimizing its patient management system. The company has recently integrated Google Gemini's Generative AI capabilities to enhance its application for real-time patient data analysis and decision support. As part of a project team, you are tasked with analyzing the current LLM applications to identify their strengths and limitations, establishing the design of algorithms that improve efficiency in processing patient data, and assessing the feasibility of these AI applications in streamlining engineering processes. Additionally, you will evaluate the performance effectiveness of the Retrieval Augmented Generation (RAG) model implemented in the system, ensuring that the solu

### Question and Answer Generation Agent for Case Study

The Question and Answer Generation Agent is designed to facilitate the creation of educational content by generating scenario-based questions and answers. This agent operates by taking in a specified Learning Outcome (LO) and its associated abilities, retrieving relevant content, and then generating questions and answers that align with the given scenario and Bloom's Taxonomy level.

#### Key Features:
1. **Input Handling**:
    - **Learning Outcome (LO)**: The specific educational goal that the learners are expected to achieve.
    - **Associated Abilities**: The skills or competencies that are linked to the Learning Outcome.

2. **Content Retrieval**:
    - The agent retrieves content that is relevant to the specified Learning Outcome and its associated abilities. This ensures that the generated questions and answers are contextually appropriate and aligned with the educational objectives.

3. **Scenario Alignment**:
    - The agent generates a realistic and practical scenario that aligns with the Learning Outcome. This scenario provides a context for the questions and answers, making them more engaging and applicable to real-world situations.

4. **Question and Answer Generation**:
    - Based on the retrieved content and the generated scenario, the agent formulates questions that require learners to demonstrate their understanding and mastery of the Learning Outcome and associated abilities.
    - The answers are crafted to align with Bloom's Taxonomy level, ensuring that they meet the desired cognitive complexity and educational standards.

5. **Output**:
    - The final output includes the scenario, the question, and the answer, all structured in a clear and concise format. This output can be used directly in educational assessments or as part of instructional materials.

By integrating these features, the Question and Answer Generation Agent ensures that the generated educational content is both relevant and effective in achieving the specified learning objectives.

In [ ]:
# from llama_index.llms.openai import OpenAI
# from pydantic.v1 import BaseModel, Field
# from typing import List
# from IPython.display import display, Markdown, Image

# # System prompt tailored for content retrieval and question-answer generation
# system_prompt = """\
# You are a content retrieval assistant. Your role is to retrieve and summarize topic content that aligns strictly with the specified Learning Outcome (LO) and its associated abilities.

# Your role:
# 1. Restrict your retrieval strictly to the specified topic provided in the query.
# 2. Retrieve and summarize the content from the topic that directly aligns with the provided Learning Outcome (LO) and its abilities.
# 3. If no specific content directly aligns with the Learning Outcome or abilities, provide a general summary of the topic instead.
# 4. Identify and extract the exact inline segments from the provided documents that directly correspond to the content used to generate the summary. The extracted segments must be verbatim snippets from the documents, ensuring a word-for-word match with the text in the provided documents.

# Ensure that:
# - (Important) Each retrieved segment is an exact match to a part of the document and is fully contained within the document text.
# - The relevance of each segment to the Learning Outcome or abilities is clear and directly supports the summary provided.
# - (Important) If you didn't use the specific document or topic, do not mention it.
# - If no relevant information is found for the Learning Outcome, clearly state this and provide a general topic summary instead.

# Restrictions:
# - Do not include content from other topics or slides outside the specified topic.
# - Each retrieved segment must explicitly belong to the given topic.
# - Avoid including assumptions or content outside the scope of the Learning Outcome and abilities.

# You must always provide:
# 1. A detailed summary of the retrieved content aligned with the Learning Outcome and abilities.
# 2. A list of verbatim extracted segments that directly support the summary, each labeled with the topic and document it belongs to.
# """

In [ ]:
# from llama_index.llms.openai import OpenAI
# from typing import List
# from IPython.display import display, Markdown

# lo_retriever_llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, system_prompt=system_prompt)

In [ ]:
# qa_generation_query_engine = index.as_query_engine(
#     similarity_top_k=10,
#     llm=lo_retriever_llm,
#     response_mode="compact",
# )

In [ ]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.cache import Cache

class CaseStudyQuestion(BaseModel):
    question: str
    answer: str
    ability_id: List[str]

class CaseStudy(BaseModel):
    scenario: str
    questions: List[CaseStudyQuestion]

user_proxy_agent = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda msg: msg.get("content", "") and "TERMINATE" in msg["content"],
    code_execution_config={"work_dir": "output", "use_docker": False}
)

# Autogen setup
qa_generation_agent = AssistantAgent(
    name="Question Answer Generator",
    system_message=f"""
    You are an expert educator in '{extracted_data.course_title}'. You will create scenario-based case study question-answer pairs based on course data.
    The data will include:
    - A scenario
    - Retrieved content aligned with learning outcomes and abilities

    ### Instructions:
    1. Use the provided scenario and retrieved content to generate one question-and-answer pairs per one learning outcome.
    2. Each question should be aligned with the learning outcome and abilities implied by the retrieved content and the Bloom's Taxonomy Level.
    3. The answer should demonstrate mastery of the abilities and address the scenario context.
    4. Ensure all keys and values are double-quoted in the JSON output.
    5. Return the output in JSON format with the following structure:
        import
        ```json
        {{
            "course_title": "<course_title_here>",
            "duration": "<assessment_duration_here>",
            "scenario": "<scenario_here>",
            "questions": [
            {{
                "question_statement": "<question_text>",
                "answer": "<answer_text>",
                "ability_id": ["<list_of_ability_ids>"]
            }},
            ...
            ]
        }}
        ```
    """,
    llm_config={
        "config_list": [
            {
                'model': "gpt-4o-mini",
                'api_key': OPENAI_API_KEY,
            },
        ],
        "timeout": 120,
    },
)
assessment_duration = ""
for assessment in extracted_data.assessments:
    if "PP" in assessment.code:
        assessment_duration = assessment.duration

with Cache.disk() as cache:
    chat_result = user_proxy_agent.initiate_chat(
        qa_generation_agent,
        message=f"""
        Please generate the questions and answer using the following course title:'{extracted_data.course_title}', assessment_duration:'{assessment_duration}', scenario: '{scenario}' and topic contents:{retrieved_content}
        Phrase your question in accordance with the Bloom's Taxonomy Level: {extracted_data.tsc_proficiency_level}
        Bloom's Taxonomy Level Information:
            Level 1: Remembering
            Level 2: Understanding
            Level 3: Applying
            Level 4: Analyzing
            Level 5: Evaluating
            Level 6: Creating
        Return the question and answer as a complete JSON dictionary containing the specified fields.
        RETURN 'TERMINATE' once the generation is done.
        """,
        summary_method="reflection_with_llm",
        cache=cache,
)


[autogen.oai.client: 12-11 15:31:47] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
user_proxy (to Question Answer Generator):


        Please generate the questions and answer using the following course title:'Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini', assessment_duration:'1 hr', scenario: 'TechSolutions Inc., a mid-sized technology consulting firm, has been experiencing significant challenges in managing client communications and project updates. With a growing portfolio of clients, the firm struggles to maintain timely and effective communication, leading to misunderstandings and delays in project deliverables. To address these issues, the management team has decided to explore the implementation of a Large Language Model (LLM) application using Google Gemini. They aim to analyze various LLM applications that can automate client interactions, streamline project upd

c:\Users\dljh1\anaconda3\envs\autogen02\Lib\site-packages\autogen\agentchat\conversable_agent.py:1258: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


In [63]:
try:
    last_message_content = chat_result.chat_history[-1].get("content", "")
    if not last_message_content:
        print("No content found in the agent's last message.")
    last_message_content = last_message_content.strip()
    json_pattern = re.compile(r'```json\s*(\{.*?\})\s*```', re.DOTALL)
    json_match = json_pattern.search(last_message_content)
    if json_match:
        json_str = json_match.group(1)
        context = json.loads(json_str)
        print(f"CONTEXT JSON MAPPING: \n\n{context}")
except json.JSONDecodeError as e:
    print(f"Error parsing context JSON: {e}")


CONTEXT JSON MAPPING: 

{'course_title': 'Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini', 'duration': '1 hr', 'scenario': 'TechSolutions Inc., a mid-sized technology consulting firm, has been experiencing significant challenges in managing client communications and project updates. With a growing portfolio of clients, the firm struggles to maintain timely and effective communication, leading to misunderstandings and delays in project deliverables. To address these issues, the management team has decided to explore the implementation of a Large Language Model (LLM) application using Google Gemini. They aim to analyze various LLM applications that can automate client interactions, streamline project updates, and enhance overall customer satisfaction. The team is particularly interested in assessing the feasibility of integrating this technology into their existing workflows and evaluating its potential to improve engineering processes and 

In [64]:
# Autogen setup
user_proxy_agent = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda msg: msg.get("content", "") and "TERMINATE" in msg["content"],
    code_execution_config={"work_dir": "output", "use_docker": False}
)

cs_writer_agent = AssistantAgent(
    name="Question Answer Generator",
    system_message="""
    You are an assistant that generates scenario-based question-answer pairs based on course data.

    Your Task:
    1. You have received the assessment information JSON dictionary.
    2. Call the `generate_documents` function with the arguments: context=context.
    **Example function call:**
    ```python
    generate_document(context=json context)
    ```
    3. Ensure that you only pass 'context' as arguments.
    4. After the function call, include the output path returned by the function in your final message, starting with `Output Paths: ` followed by the path.
    5. Return 'TERMINATE' when the task is done.
    """,
    llm_config={
        "config_list": [
            {
                'model': "gpt-4o-mini",
                'api_key': OPENAI_API_KEY,
            },
        ],
        "timeout": 120,
    },
)

@user_proxy_agent.register_for_execution()
@cs_writer_agent.register_for_llm(name="generate_document", description="Generate the Facilitator Guide document")
def generate_documents(context: dict, output_dir: str) -> str:
    """
    Generate the question paper and answer paper for the given context and type.

    Parameters:
    - context (dict): The data for the assessment (course title, type, questions, etc.).
    - type (int): The assessment type (1 for Ability-based, 2 for Knowledge-based).
    - output_dir (str): Directory where the generated documents will be saved.

    Returns:
    - dict: Paths to the generated documents (question and answer papers).
    """
    # Define template paths
    TEMPLATES = {
        1: { # PP
            "ANSWER": "../Templates/(Template) Answer to A Assessment - Course Title - v1.docx",
            "QUESTION": "../Templates//(Template) A Assessment - Course Title - v1.docx"
        },
        2: { # SAQ
            "ANSWER": "../Templates/(Template) Answer to WA (SAQ) - Course Title - v1.docx",
            "QUESTION": "../Templates/(Template) WA (SAQ) - Course Title - v1.docx"
        },
        3: { # CS
            "ANSWER": "../Templates/(Template) Answers to CS - Course Title - v1.docx",
            "QUESTION": "../Templates/(Template) CS - Course Title - v1.docx"
        },
    }

    # Validate type
    if type not in TEMPLATES:
        raise ValueError("Invalid type. Must be 1 (Ability-based) or 2 (Knowledge-based).")

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load templates
    answer_template_path = TEMPLATES[3]["ANSWER"]
    question_template_path = TEMPLATES[3]["QUESTION"]
    answer_doc = DocxTemplate(answer_template_path)
    question_doc = DocxTemplate(question_template_path)

    # Prepare context for the question paper by creating a copy of the context without answers
    question_context = {
        **context,
        "questions": [
            {
                **question,
                "answer": None,  # Remove answers for the question document
            }
            for question in context.get("questions", [])
        ]
    }

    # Render both templates
    answer_doc.render(context)  # Render with answers
    question_doc.render(question_context)  # Render without answers

    # Save the documents to the output directory
    files = {
        "ANSWER": os.path.join(output_dir, f"Answers to CS - {context['course_title']} - v1.docx"),
        "QUESTION": os.path.join(output_dir, f"CS - {context['course_title']} - v1.docx")
    }
    answer_doc.save(files["ANSWER"])
    question_doc.save(files["QUESTION"])

    return files  # Return paths to the generated documents

with Cache.disk() as cache:
    chat_result = user_proxy_agent.initiate_chat(
        qa_generation_agent,
        message=f"""
        Please generate assessment documents with the context: {context}
        RETURN 'TERMINATE' once the document generation is done.
        """,
        summary_method="reflection_with_llm",
        cache=cache,
)



[autogen.oai.client: 12-11 15:32:59] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 12-11 15:32:59] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
user_proxy (to Question Answer Generator):


        Please generate assessment documents with the context: {'course_title': 'Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini', 'duration': '1 hr', 'scenario': 'TechSolutions Inc., a mid-sized technology consulting firm, has been experiencing significant challenges in managing client communications and project updates. With a growing portfolio of clients, the firm struggles to maintain timely and effective communication, leading to misunderstandings and delays in project deliverables. To address these issues, the management team has decided to explore the implementation of a Large Language Model (LLM

In [67]:
from docxtpl import DocxTemplate

def generate_documents(context: dict, type: int, output_dir: str) -> dict:
    """
    Generate the question paper and answer paper for the given context and type.

    Parameters:
    - context (dict): The data for the assessment (course title, type, questions, etc.).
    - type (int): The assessment type (1 for Ability-based, 2 for Knowledge-based).
    - output_dir (str): Directory where the generated documents will be saved.

    Returns:
    - dict: Paths to the generated documents (question and answer papers).
    """
    # Define template paths
    TEMPLATES = {
        1: { # PP
            "ANSWER": "../Templates/(Template) Answer to A Assessment - Course Title - v1.docx",
            "QUESTION": "../Templates//(Template) A Assessment - Course Title - v1.docx"
        },
        2: { # SAQ
            "ANSWER": "../Templates/(Template) Answer to WA (SAQ) - Course Title - v1.docx",
            "QUESTION": "../Templates/(Template) WA (SAQ) - Course Title - v1.docx"
        },
        3: { # CS
            "ANSWER": "../Templates/(Template) Answer to CS - Course Title - v1.docx",
            "QUESTION": "../Templates/(Template) CS - Course Title - v1.docx"
        },
        # 4: { # OQ
        #     "ANSWER": None,
        #     "QUESTION": r"C:\Users\dljh1\Documents\courseware_autogen\Assessment\Templates\(Template) WA (SAQ) - Course Title - v1.docx"
        # },
    }

    # Validate type
    if type not in TEMPLATES:
        raise ValueError("Invalid type. Must be 1 (Ability-based) or 2 (Knowledge-based).")

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load templates
    answer_template_path = TEMPLATES[type]["ANSWER"]
    question_template_path = TEMPLATES[type]["QUESTION"]
    answer_doc = DocxTemplate(answer_template_path)
    question_doc = DocxTemplate(question_template_path)

    # Prepare context for the question paper by creating a copy of the context without answers
    question_context = {
        **context,
        "questions": [
            {
                **question,
                "answer": None,  # Remove answers for the question document
            }
            for question in context.get("questions", [])
        ]
    }

    # Render both templates
    answer_doc.render(context)  # Render with answers
    question_doc.render(question_context)  # Render without answers

    # Save the documents to the output directory
    files = {
        "ANSWER": os.path.join(output_dir, f"Answer to CS - {context['course_title']} - v1.docx"),
        "QUESTION": os.path.join(output_dir, f"CS - {context['course_title']} - v1.docx")
    }
    answer_doc.save(files["ANSWER"])
    question_doc.save(files["QUESTION"])

    return files  # Return paths to the generated documents

files = generate_documents(context, type=3, output_dir="output")  # 'type=2' as per your code for knowledge-based assessment
print("Documents generated:", files)


Documents generated: {'ANSWER': 'output\\Answer to CS - Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini - v1.docx', 'QUESTION': 'output\\CS - Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini - v1.docx'}


## Generating Short Answer Questions (SAQ)

In this section, we generate SAQs for each Knowledge Statement using OpenAI. The generated questions include a scenario, a question, and an answer, which are structured in a JSON format. The process involves querying the OpenAI model with the relevant content retrieved for each Knowledge Statement. The results are then saved to a file for further use.

In [1]:
from llama_index.llms.openai import OpenAI
from typing import List

# System prompt tailored for content retrieval
system_prompt = """\
You are a content retrieval assistant tasked with retrieving educational topic content aligned with a given Knowledge Statement.

Your role:
1. Restrict your retrieval strictly to the specified topic provided in the query.
2. Retrieve and summarize the topic content that aligns with the provided Knowledge Statement.
3. If no specific content directly aligns with the Knowledge Statement, provide a general summary of the specified topic instead.
4. Identify and extract the exact inline segments from the provided documents that directly correspond to the content used to 
generate the given answer. The extracted segments must be verbatim snippets from the documents, ensuring a word-for-word match with the text 
in the provided documents.

Ensure that:
- (Important) Each segment is an exact match to a part of the document and is fully contained within the document text.
- The relevance of each segment to the generated answer is clear and directly supports the answer provided.
- (Important) If you didn't used the specific document don't mention it.
- If no relevant information is found for the Knowledge Statement, clearly state this and provide a general topic summary instead.

Restrictions:
- Do not include content from other topics or slides outside the specified topic.
- Each retrieved segment must explicitly belong to the given topic.
"""

llm = OpenAI(model="gpt-4o-mini", system_prompt=system_prompt)

In [4]:
query_engine = index.as_query_engine(
    similarity_top_k=10,
    llm=llm,
    # response_mode="tree_summarize"
    response_mode="compact",
)

In [7]:
# Initialize a list to store structured outputs
retrieved_data: List[KnowledgeStatementContent] = []

# Iterate through all learning units and topics to query for all Knowledge Statements
for learning_unit in extracted_data.learning_units:
    for topic in learning_unit.topics:
        for knowledge in topic.tsc_knowledges:
            knowledge_id = knowledge.id
            knowledge_statement = knowledge.text
            topic_name = topic.name

            print(f"\n--- Retrieving for Knowledge Statement: {knowledge_id}: {knowledge_statement}")
            
            # Query the index to retrieve topic content for this Knowledge Statement
            response = query_engine.query(
                f"Generate questions and answer content for the following topic: '{topic.name}' aligning to the Knowledge Statement: '{knowledge_statement}'."
            )
            retrieved_content = response.response

            # Add the structured data using Pydantic model
            try:
                knowledge_data = KnowledgeStatementContent(
                    knowledge_id=knowledge_id,
                    knowledge_statement=knowledge_statement,
                    topic_name=topic_name,
                    retrieved_content=retrieved_content
                )
                retrieved_data.append(knowledge_data)
            except Exception as e:
                print(f"Error adding structured data for {knowledge_id}: {e}")

# Print structured data for verification
for entry in retrieved_data:
    print(entry.json(indent=4))

NameError: name 'extracted_data' is not defined

In [ ]:
OPENAI_API_KEY = os.getenv("TERTIARY_INFOTECH_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# Function to generate WSQ using OpenAI
def generate_wsq_openai(knowledge_id: str, knowledge_statement: str, topic_name: str, retrieved_content: str) -> WSQ:
    """
    Generate a Workplace Scenario Question (WSQ) using OpenAI.

    Args:
        knowledge_id (str): The ID of the Knowledge Statement.
        knowledge_statement (str): The Knowledge Statement.
        topic_name (str): The name of the topic.
        retrieved_content (str): The content retrieved for the Knowledge Statement.

    Returns:
        WSQ: A structured WSQ object.
    """
    messages = [
        {
            "role": "system",
            "content": "You are an experienced instructional designer tasked with creating Workplace Scenario Questions (WSQs). Your output must include a Knowledge Statement, Scenario, Question, and Answer in JSON format."
        },
        {
            "role": "user",
            "content": (
                f"Generate a WSQ for the following inputs:\n\n"
                f"Knowledge ID: {knowledge_id}\n"
                f"Knowledge Statement: {knowledge_statement}\n"
                f"Topic Name: {topic_name}\n"
                f"Retrieved Content: {retrieved_content}\n\n"
                f"Ensure the response adheres to this schema: {{'knowledge_id': 'string', 'knowledge_statement': 'string', 'scenario': 'string', 'question': 'string', 'answer': 'string'}}."
            )
        }
    ]

    # Call OpenAI's structured response API
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=messages,
        response_format=WSQ
    )

    try:
        wsq_data = response.choices[0].message.parsed
        return wsq_data
    except Exception as e:
        print(f"Error generating WSQ: {e}")
        return None


In [22]:
# Generate WSQs for each Knowledge Statement in retrieved_data
wsq_results = []

for entry in retrieved_data:
    # Access attributes of the Pydantic model
    knowledge_id = entry.knowledge_id
    knowledge_statement = entry.knowledge_statement
    topic_name = entry.topic_name
    retrieved_content = entry.retrieved_content

    # Generate WSQ
    wsq = generate_wsq_openai(knowledge_id, knowledge_statement, topic_name, retrieved_content)
    if wsq:
        print(f"\n--- WSQ for {knowledge_id}: {knowledge_statement} ---\n")
        print(f"Scenario: {wsq.scenario}\n")
        print(f"Question: {wsq.question}\n")
        print(f"Answer: {wsq.answer}\n")
        wsq_results.append(wsq)

# Save WSQs to a file for further use
with open("wsq_results.json", "w") as f:
    json.dump([wsq.dict() for wsq in wsq_results], f, indent=4)


--- WSQ for K1: Range of AI applications ---

Scenario: As a business analyst, you are tasked with presenting a report on the potential applications of Large Language Models (LLMs) to your team. You have identified several areas where LLMs can be beneficial, such as customer support, content creation, and educational tools.

Question: What are some common applications of Large Language Models (LLMs) that you could include in your report to the team?

Answer: Common applications of LLMs include content creation and assistance, customer support and chatbots, language translation and localization, educational tools, business intelligence and analytics, accessibility for disabled persons, coding and development, legal and compliance assistance, healthcare support, art and design inspiration, enhanced search engines, and crisis management and response.


--- WSQ for K6: Applicability of AI in the industry ---

Scenario: A manager at a healthcare organization is exploring new technologies t

In [23]:
print(wsq_results)

[WSQ(knowledge_id='K1', knowledge_statement='Range of AI applications', scenario='As a business analyst, you are tasked with presenting a report on the potential applications of Large Language Models (LLMs) to your team. You have identified several areas where LLMs can be beneficial, such as customer support, content creation, and educational tools.', question='What are some common applications of Large Language Models (LLMs) that you could include in your report to the team?', answer='Common applications of LLMs include content creation and assistance, customer support and chatbots, language translation and localization, educational tools, business intelligence and analytics, accessibility for disabled persons, coding and development, legal and compliance assistance, healthcare support, art and design inspiration, enhanced search engines, and crisis management and response.'), WSQ(knowledge_id='K6', knowledge_statement='Applicability of AI in the industry', scenario='A manager at a he

In [28]:
for wsq in wsq_results:
    print(wsq.dict())

{'knowledge_id': 'K1', 'knowledge_statement': 'Range of AI applications', 'scenario': 'As a business analyst, you are tasked with presenting a report on the potential applications of Large Language Models (LLMs) to your team. You have identified several areas where LLMs can be beneficial, such as customer support, content creation, and educational tools.', 'question': 'What are some common applications of Large Language Models (LLMs) that you could include in your report to the team?', 'answer': 'Common applications of LLMs include content creation and assistance, customer support and chatbots, language translation and localization, educational tools, business intelligence and analytics, accessibility for disabled persons, coding and development, legal and compliance assistance, healthcare support, art and design inspiration, enhanced search engines, and crisis management and response.'}
{'knowledge_id': 'K6', 'knowledge_statement': 'Applicability of AI in the industry', 'scenario': 'A

In [ ]:
from docxtpl import DocxTemplate
import os

def generate_documents(context: dict, type: int, output_dir: str) -> dict:
    """
    Generate the question paper and answer paper for the given context and type.

    Parameters:
    - context (dict): The data for the assessment (course title, type, questions, etc.).
    - type (int): The assessment type (1 for Ability-based, 2 for Knowledge-based).
    - output_dir (str): Directory where the generated documents will be saved.

    Returns:
    - dict: Paths to the generated documents (question and answer papers).
    """
    # Define template paths
    TEMPLATES = {
        1: { # PP
            "ANSWER": "../Templates/(Template) Answer to A Assessment - Course Title - v1.docx",
            "QUESTION": "../Templates//(Template) A Assessment - Course Title - v1.docx"
        },
        2: { # SAQ
            "ANSWER": "../Templates/(Template) Answer to WA (SAQ) - Course Title - v1.docx",
            "QUESTION": "../Templates/(Template) WA (SAQ) - Course Title - v1.docx"
        },
        3: { # CS
            "ANSWER": "../Templates/(Template) Answers to CS - Course Title - v1.docx",
            "QUESTION": "../Templates/(Template) CS - Course Title - v1.docx"
        },
        # 4: { # OQ
        #     "ANSWER": None,
        #     "QUESTION": r"C:\Users\dljh1\Documents\courseware_autogen\Assessment\Templates\(Template) WA (SAQ) - Course Title - v1.docx"
        # },
    }

    # Validate type
    if type not in TEMPLATES:
        raise ValueError("Invalid type. Must be 1 (Ability-based) or 2 (Knowledge-based).")

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load templates
    answer_template_path = TEMPLATES[type]["ANSWER"]
    question_template_path = TEMPLATES[type]["QUESTION"]
    answer_doc = DocxTemplate(answer_template_path)
    question_doc = DocxTemplate(question_template_path)

    # Prepare context for the question paper by creating a copy of the context without answers
    question_context = {
        **context,
        "questions": [
            {
                **question,
                "answer": None,  # Remove answers for the question document
            }
            for question in context.get("questions", [])
        ]
    }

    # Render both templates
    answer_doc.render(context)  # Render with answers
    question_doc.render(question_context)  # Render without answers

    # Save the documents to the output directory
    files = {
        "ANSWER": os.path.join(output_dir, f"Answers to WA(SAQ) - {context['course_title']} - v1.docx"),
        "QUESTION": os.path.join(output_dir, f"WA(SAQ) - {context['course_title']} - v1.docx")
    }
    answer_doc.save(files["ANSWER"])
    question_doc.save(files["QUESTION"])

    return files  # Return paths to the generated documents

In [ ]:
for assessment in extracted_data.assessments:
    if 'SAQ' in assessment.code:
        context = {
            "course_title": extracted_data.course_title,  # e.g., extracted_data.course_title
            "duration": assessment.duration,
            # The scenario field at the top level can be a general overview or introduction.
            # Since each WSQ also has its own scenario, you can choose to put a general scenario here:
            "scenario": "Below are scenario-based questions derived from the course content and Knowledge Statements.",
            "questions": []
        }

        # Populate the questions from wsq_results
        for wsq in wsq_results:
            # Append the WSQ details along with the knowledge ID to the context
            context["questions"].append({
                "knowledge_id": wsq.knowledge_id,
                "scenario": wsq.scenario,
                "question_statement": wsq.question,
                "answer": wsq.answer
            })

        print("Context prepared for document generation.")

        files = generate_documents(context, type=2, output_dir="output")  # 'type=2' as per your code for knowledge-based assessment
        print("Documents generated:", files)

Context prepared for document generation.
Documents generated: {'ANSWER': 'output\\Answers to WA(SAQ) - Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini - v1.docx', 'QUESTION': 'output\\WA(SAQ) - Develop Artificial Intelligence and Large Language Model (LLM) Applications with Google Gemini - v1.docx'}
